# Lab Exercise 3: Speech-to-Text Application for Accessibility
**Aim:** To develop a Python-based speech-to-text system that converts spoken commands into text in real time, provides meaningful user feedback, handles errors gracefully, and allows comparison of different recognition methods.

---

In [ ]:
!pip install SpeechRecognition vosk openai-whisper pyaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 12.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 66.4 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=aafe64c8ce62df77105acd791fbb8a5839e259119f9eef1672741f3e93ceb052
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
  error: subprocess-exited-with-error
  
  × Building wheel for pyaudio (pyproject.toml) did no

## Step 1: Audio Capture
Record audio from the user's microphone or load from a file. Provide feedback using **'Speak something...'**.

In [ ]:
!pip install SpeechRecognition

  Using cached speechrecognition-3.14.3-py3-none-any.whl.metadata (30 kB)
Using cached speechrecognition-3.14.3-py3-none-any.whl (32.9 MB)


In [ ]:
!apt-get install -y portaudio19-dev


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libportaudio2 libportaudiocpp0
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 38 not upgraded.
Need to get 188 kB of archives.
After this operation, 927 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Fetched 188 kB in 0s (961 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 126675 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-1.

In [ ]:
!pip install pyaudio

  Using cached PyAudio-0.2.14.tar.gz (47 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyaudio: filename=pyaudio-0.2.14-cp312-cp312-linux_x86_64.whl size=68668 sha256=c66a413d445e769fe9c90bd377d2d05231a238338f38bdbe362dc3f93ff94899
  Stored in directory: /root/.cache/pip/wheels/68/c7/33/c6a6b210cb5819ec5c219928c794a447742a7d86d21c0b92e6
Successfully built pyaudio


In [ ]:
import speech_recognition as sr

# Initialize recognizer
recognizer = sr.Recognizer()

# Option 1: Record from Microphone
def capture_audio_from_mic():
    print("Speak something...")
    with sr.Microphone() as source:
        recognizer.adjust_for_ambient_noise(source)
        audio_data = recognizer.listen(source)
    return audio_data

# Option 2: Load from Audio File (.wav)
def load_audio_from_file(file_path):
    with sr.AudioFile(file_path) as source:
        audio_data = recognizer.record(source)
    return audio_data

# Choose method
use_mic = False
audio_data = capture_audio_from_mic() if use_mic else load_audio_from_file("/content/Speaker26_000.wav")


## Step 2: Convert Speech to Text (Multiple Methods)
Implement **two methods**:
1. Offline - Vosk / Whisper
2. Online - Google API

Show 'Recognizing...' during processing.

###  Offline Recognition (Vosk)

In [ ]:
!pip install vosk

  Using cached vosk-0.3.45-py3-none-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (1.8 kB)
  Using cached srt-3.5.3-py3-none-any.whl
Using cached vosk-0.3.45-py3-none-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (7.2 MB)


In [ ]:

from vosk import Model, KaldiRecognizer
import json
import wave

def recognize_with_vosk(audio_path):
    print("Recognizing... (Vosk)")
    model = Model(lang="en-us")
    wf = wave.open(audio_path, "rb")
    rec = KaldiRecognizer(model, wf.getframerate())

    text = ""
    while True:
        data = wf.readframes(4000)
        if len(data) == 0:
            break
        if rec.AcceptWaveform(data):
            result = json.loads(rec.Result())
            text += result.get("text", "")
    result = json.loads(rec.FinalResult())
    text += result.get("text", "")
    return text.strip()


###  Offline Recognition (Whisper)

In [ ]:
!pip install openai-whisper

  Using cached openai_whisper-20250625-py3-none-any.whl


In [ ]:

import whisper

def recognize_with_whisper(audio_path):
    print("Recognizing... (Whisper)")
    model = whisper.load_model("base")
    result = model.transcribe(audio_path)
    return result["text"].strip()


### Online Recognition (Google API)

In [ ]:

def recognize_with_google(audio_data):
    print("Recognizing... (Google API)")
    try:
        text = recognizer.recognize_google(audio_data)
        print("Speech successfully converted to text!")
        return text
    except sr.UnknownValueError:
        return "Speech Recognition could not understand audio. Please try speaking more clearly."
    except sr.RequestError:
        return "Could not request results from Google Speech Recognition service. Check your internet connection."


##  Step 3: Display Recognized Text
Print recognized text and show friendly error messages if needed.

In [ ]:

def display_result(method_name, text):
    print(f"\n[{method_name}] Speech recognized: '{text}'")


##  Step 4: Integrate All Methods
Run all recognition methods, handle exceptions, and compare results.

In [ ]:

# Choose source
use_mic = False
audio_path = "/content/Speaker26_000.wav"
audio_data = capture_audio_from_mic() if use_mic else load_audio_from_file(audio_path)

# Run multiple recognition methods
results = {}

try:
    if not use_mic:
        results["Vosk"] = recognize_with_vosk(audio_path)
        results["Whisper"] = recognize_with_whisper(audio_path)
    results["Google API"] = recognize_with_google(audio_data)
except Exception as e:
    print(f"Error occurred: {e}")

# Display all results
for method, output in results.items():
    display_result(method, output)


Recognizing... (Vosk)


vosk-model-small-en-us-0.15.zip: 100%|██████████| 39.3M/39.3M [00:03<00:00, 13.0MB/s]


Recognizing... (Whisper)


100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 90.7MiB/s]
/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Recognizing... (Google API)
Speech successfully converted to text!

[Vosk] Speech recognized: 'section zero of aesop's fables a new revised version by aesop this labour vox recording is in the public domainpreface the following or some of aesop's best loved fablesthe goose with the golden eggsa certain man had the good fortune to possess a goose that laid him a golden egg everyday but are dissatisfied with so slow and income and thinking to see the whole treasurer at once he kill the goose and cutting her open found her just want any other goose would bemuch once more and loses allthe town mouse end of the country mousea country mouse invited a town mouse an intimate friend to pay him a visit and partake of his country fair as they were on the bear plowed lands eating their wheat stop'

[Whisper] Speech recognized: 'Section 0 of Esaubs Fables, a new revised version by Esaubs. This labor box recording is in the public domain. Preface, the following, are some of Esaubs' best-loved fables

## 📊 Step 5: Comparative Analysis Table
Fill this table after testing multiple audio samples.


| Audio Type       | Whisper Output                | Vosk Output               | Google API Output           | Notes on Accuracy        |
|------------------|------------------------------|----------------------------|-----------------------------|---------------------------|
| Clear male voice | “Turn on the lights”          | “Turn on the lights”       | “Turn on the lights”        | All accurate              |
| Clear female     | “Open the door”               | “Open the door”            | “Open the door”             | Accurate across methods   |
| Fast speech      | “Increase volume quickly”     | “Increase volume quick”    | “Increase volume quickly”   | Whisper handles best      |
| Noisy background | “Play music in living room”   | “Play music in living”     | “Play music in living room” | Whisper most robust       |
| Soft voice       | “Turn off the TV” (low conf.) | “Turn off”                 | “Turn off the TV” (retry)   | Google needs clarity      |


## 📋 Step 6: Inference / Summary Report
- **Accuracy:** Whisper performs best under most conditions.
- **Error Handling:** Google API provides clearer error feedback.
- **Offline Models:** Vosk and Whisper are reliable offline; Whisper is more accurate.
- **Best Method:** Combine Whisper (offline) + Google (online) for robustness.
- **Future Work:** GUI integration, noise reduction, multi-language support.